In [1]:
import sys
import os
import json

current_dir = os.getcwd()
print("Current Directory:", current_dir)

sys.path.insert(0, os.path.abspath(os.path.join(current_dir, '..', 'src')))

Current Directory: /home/stevewang/Github/look-it-from-here/notebooks


In [2]:
from playwright.async_api import async_playwright
from look_it_from_here.core.transform import create_html_tree, create_semantic_tree, create_embeddings_from_semantic_tree
from look_it_from_here.adapter.playwright_implementation import PlaywrightPage
from look_it_from_here.core.embeddings import Embedder
from look_it_from_here.core.element_selector import ElementSelector

In [3]:
p = await async_playwright().start()
browser = await p.chromium.launch(headless=False)
page = await browser.new_page()

await page.goto("https://www.google.com")
await page.wait_for_load_state("networkidle")

pw_page = PlaywrightPage(page)

## Direct Testing - Step by Step

In [4]:
# Step 1: Create HTML tree
html_tree, dom_id_to_webelement = await create_html_tree(pw_page)
print(f"HTML tree: {html_tree.tag}")
print(f"DOM elements mapped: {len(dom_id_to_webelement)}")

HTML tree: html
DOM elements mapped: 436


In [5]:
# Step 2: Create semantic tree  
semantic_tree, dom_id_to_semantic_id = create_semantic_tree(html_tree)
print(f"Semantic tree: {semantic_tree.tag}")
print(f"DOM to semantic mappings: {len(dom_id_to_semantic_id)}")

Semantic tree: body
DOM to semantic mappings: 117


In [ ]:
# Step 3: Create embeddings and reverse trees
embedder = Embedder()
semantic_id_to_embedding, semantic_id_to_reverse_tree = create_embeddings_from_semantic_tree(semantic_tree, embedder)
print(f"Embeddings created: {len(semantic_id_to_embedding)}")
print(f"Reverse trees created: {len(semantic_id_to_reverse_tree)}")

In [ ]:
# Debug: Look at reverse tree representations
print("\\nSample reverse trees:")
for i, (semantic_id, reverse_tree) in enumerate(semantic_id_to_reverse_tree.items()):
    if i >= 3:  # Show first 3
        break
    print(f"\\n--- {semantic_id} ---")
    print(reverse_tree.to_text()[:200] + "...")

In [ ]:
# Step 4: Create semantic_id_to_webelement mapping
semantic_id_to_webelement = {}
for dom_id, semantic_id in dom_id_to_semantic_id.items():
    if dom_id in dom_id_to_webelement:
        semantic_id_to_webelement[semantic_id] = dom_id_to_webelement[dom_id]

print(f"Semantic to web element mappings: {len(semantic_id_to_webelement)}")

In [ ]:
# Step 5: Test element selection directly
class MockSnapshot:
    def __init__(self):
        self.semantic_id_to_embedding = semantic_id_to_embedding
        self.semantic_id_to_webelement = semantic_id_to_webelement

mock_snapshot = MockSnapshot()
selector = ElementSelector(embedder)

results = selector.select_elements(mock_snapshot, "Images button", top_k=3)
print("Search results:")
for semantic_id, element, score in results:
    print(f"  {semantic_id}: {score:.4f}")

In [ ]:
# Test different queries
queries = ["search button", "navigation", "google logo", "lucky button"]

for query in queries:
    results = selector.select_elements(mock_snapshot, query, top_k=2)
    print(f"\n'{query}':")
    for semantic_id, element, score in results:
        print(f"  {score:.4f} - {semantic_id}")

In [ ]:
# Cleanup
await browser.close()
await p.stop()